Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [0]:
!git clone https://github.com/nshepperd/gpt-2.git
#!git clone https://github.com/mowillia/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 297, done.
remote: Total 297 (delta 0), reused 0 (delta 0), pack-reused 297
Receiving objects: 100% (297/297), 4.40 MiB | 14.71 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [0]:
cd gpt-2

/content/gpt-2


In [0]:
#check GPU status
!nvidia-smi

Thu Jun 27 15:42:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
# Checking if GPU is available
work_with_gpu = torch.cuda.is_available()
if(work_with_gpu):
    print('Using GPU!')
else: 
    print('No GPU available, using CPU; Consider using short texts.')

Using GPU!


Install requirements

In [0]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 604kB 9.7MB/s 
     |████████████████████████████████| 51kB 23.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Download the model data

In [0]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 566kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 54.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 923kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 52.1Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.53Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 53.5Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 52.4Mit/s]                                                       


In [0]:
#!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 814kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 54.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.02Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:21, 65.4Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.13Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 52.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 46.1Mit/s]                                                       


encoding

In [0]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [0]:
# Get full essays
!cp -r /content/drive/My\ Drive/writrly_proj_files/Full_Essays/* /content/

In [0]:
# Fetch the models 
#!cp -r /content/drive/My\ Drive/models/ /content/gpt-2/ 

In [0]:
# Fetch the checkpoints 
#!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 


Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [0]:
#!wget https://www.gutenberg.org/files/98/98-0.txt


Start training, add --model_name '345M' to use 345 model

### Training without concatenation

In [0]:
## business essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_business.txt --run_name 'atlantic_business_345' --model_name '345M'

W0627 15:46:08.758351 140659569104768 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 15:46:08.766982 140659569104768 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 15:46:08.857946 140659569104768 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 15:46:08.858297 140659569104768 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 15:46:08.876915: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 15:46:08.879284: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d65100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## technology essays training  - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_technology.txt --run_name 'atlantic_technology_345' --model_name '345M'

W0627 16:18:07.355815 140453592807296 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 16:18:07.364850 140453592807296 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 16:18:07.454462 140453592807296 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 16:18:07.454832 140453592807296 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 16:18:07.461376: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 16:18:07.461646: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1803100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## science essays training  - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_science.txt --run_name 'atlantic_science_345' --model_name '345M'

W0627 16:50:15.011774 140357754636160 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 16:50:15.029707 140357754636160 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 16:50:15.118153 140357754636160 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 16:50:15.118532 140357754636160 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 16:50:15.125158: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 16:50:15.125436: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x135f100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## education essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_education.txt --run_name 'atlantic_education_345' --model_name '345M'

W0627 17:22:20.994460 139710809851776 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 17:22:21.003343 139710809851776 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 17:22:21.095731 139710809851776 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 17:22:21.096116 139710809851776 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 17:22:21.102430: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 17:22:21.102697: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x15eb100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## politics essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_politics.txt --run_name 'atlantic_politics_345' --model_name '345M'

W0627 17:54:28.311069 140594242262912 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 17:54:28.319618 140594242262912 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 17:54:28.410823 140594242262912 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 17:54:28.411276 140594242262912 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 17:54:28.417679: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 17:54:28.417938: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1889100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## entertainment essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_entertainment.txt --run_name 'atlantic_entertainment_345' --model_name '345M'

W0627 18:26:38.688606 139911907395456 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 18:26:38.697175 139911907395456 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 18:26:38.784096 139911907395456 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 18:26:38.784461 139911907395456 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 18:26:38.790568: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 18:26:38.790825: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1361100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## ideas essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_ideas.txt --run_name 'atlantic_ideas_345' --model_name '345M'

W0627 18:58:51.886311 140366815840128 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 18:58:51.895255 140366815840128 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 18:58:51.986181 140366815840128 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 18:58:51.986577 140366815840128 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 18:58:51.993001: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 18:58:51.993257: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1ce7100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## international essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_international.txt --run_name 'atlantic_international_345' --model_name '345M'

W0627 19:31:16.338305 139982513698688 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 19:31:16.348274 139982513698688 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 19:31:16.450475 139982513698688 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 19:31:16.450879 139982513698688 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 19:31:16.457513: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 19:31:16.457789: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x29d7100 executing computations on platform Host. Devices:
2019-06-27 1

In [0]:
## health essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_health.txt --run_name 'atlantic_health_345' --model_name '345M'

W0627 20:03:50.572252 140538845390720 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 20:03:50.580771 140538845390720 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 20:03:50.676445 140538845390720 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 20:03:50.676766 140538845390720 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 20:03:50.683209: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 20:03:50.683482: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2dc9100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## gutenberg training
!PYTHONPATH=src ./train.py --dataset /content/all_gutenberg.txt --run_name 'gutenberg_345' --model_name '345M'

W0627 20:36:30.005469 140712750081920 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 20:36:30.015077 140712750081920 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 20:36:30.110814 140712750081920 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 20:36:30.111193 140712750081920 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 20:36:30.117927: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 20:36:30.118206: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3065100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## all atlantic training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_atlantic.txt --run_name 'all_atlantic_345' --model_name '345M'

W0627 21:09:38.834139 140190179219328 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 21:09:38.844184 140190179219328 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 21:09:38.949961 140190179219328 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 21:09:38.950379 140190179219328 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 21:09:38.957622: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 21:09:38.957924: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1dfd100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## personal essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_my_journals.txt --run_name 'personal_essays_345' --model_name '345M'

W0627 21:42:35.603564 140100349421440 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 21:42:35.612083 140100349421440 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 21:42:35.710467 140100349421440 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 21:42:35.710817 140100349421440 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 21:42:35.717078: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 21:42:35.717354: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f6d100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## slate essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_slate.txt --run_name 'all_slate_345' --model_name '345M'

W0627 22:15:08.971491 139862786262912 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 22:15:08.980223 139862786262912 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 22:15:09.071049 139862786262912 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 22:15:09.071394 139862786262912 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 22:15:09.077538: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 22:15:09.077791: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2d91100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## national review essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_national_review.txt --run_name 'all_national_review_345' --model_name '345M'

W0627 22:47:37.889118 139705169528704 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 22:47:37.898118 139705169528704 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 22:47:37.996006 139705169528704 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 22:47:37.996489 139705169528704 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 22:47:38.004039: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 22:47:38.004286: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d61100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## new york post essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_nypost.txt --run_name 'all_nypsot_345' --model_name '345M'

W0627 23:20:11.072258 139997260420992 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0627 23:20:11.080600 139997260420992 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0627 23:20:11.173804 139997260420992 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0627 23:20:11.174168 139997260420992 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-27 23:20:11.180492: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-27 23:20:11.180759: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1595100 executing computations on platform Host. Devices:
2019-06-27 2

In [0]:
## short story training - with 345
!PYTHONPATH=src ./train.py --dataset /content/all_short_stories.txt --run_name 'all_short_stories_345' --model_name '345M'

W0628 01:48:52.913891 139766784378752 deprecation_wrapper.py:119] From /content/gpt-2/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0628 01:48:52.936126 139766784378752 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0628 01:48:53.018906 139766784378752 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0628 01:48:53.019173 139766784378752 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-28 01:48:53.129439: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-28 01:48:53.133149: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2a65100 executing computations on platform Host. Devices:
2019-06-28 01:48

In [0]:
## short story training 
!PYTHONPATH=src ./train.py --dataset /content/all_short_stories.txt --run_name 'all_short_stories' --model_name '117M'

W0630 04:21:27.512420 140328742782848 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0630 04:21:27.527328 140328742782848 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0630 04:21:27.613547 140328742782848 deprecation_wrapper.py:119] From ./train.py:87: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0630 04:21:27.613832 140328742782848 deprecation_wrapper.py:119] From ./train.py:90: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-30 04:21:27.631013: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-30 04:21:27.632788: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17152c0 executing computations on platform Host. Devices:
2019-06-30 0

### Training with concatenation

In [0]:
## science essays training  - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/business --combine 500 --run_name 'atlantic_business_concat_345' --model_name '345M'

W0626 13:39:02.292997 140488384042880 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0626 13:39:02.301703 140488384042880 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0626 13:39:02.395063 140488384042880 deprecation_wrapper.py:119] From ./train.py:87: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0626 13:39:02.395398 140488384042880 deprecation_wrapper.py:119] From ./train.py:90: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-26 13:39:02.420173: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-26 13:39:02.422288: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1329100 executing computations on platform Host. Devices:
2019-06-26 1

In [0]:
## technology essays training  - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/technology --combine 500 --run_name 'atlantic_technology_concat_345' --model_name '345M'

W0626 14:13:30.483827 139983816439680 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0626 14:13:30.508350 139983816439680 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0626 14:13:30.612340 139983816439680 deprecation_wrapper.py:119] From ./train.py:87: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0626 14:13:30.612653 139983816439680 deprecation_wrapper.py:119] From ./train.py:90: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-26 14:13:30.618807: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-26 14:13:30.619067: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x283b100 executing computations on platform Host. Devices:
2019-06-26 1

In [0]:
## science essays training  - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/science --combine 5000 --run_name 'atlantic_science_concat_345' --model_name '345M'

In [0]:
## education essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/education --combine 500  --run_name 'atlantic_education_concat_345' --model_name '345M'

In [0]:
## politics essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My Drive/writrly_proj_files/Atlantic_essays/politics --combine 500 --run_name 'atlantic_politics_concat_345' --model_name '345M'

In [0]:
## entertainment essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/entertainment --combine 500 --run_name 'atlantic_entertainment_concat_345' --model_name '345M'

In [0]:
## ideas essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My\ Drive/writrly_proj_files/Atlantic_essays/ideas --combine 500 --run_name 'atlantic_ideas_concat_345' --model_name '345M'

In [0]:
## international essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My Drive/writrly_proj_files/Atlantic_essays/international --combine 500 --run_name 'atlantic_international_concat_345' --model_name '345M'

In [0]:
## health essays training - with 345
!PYTHONPATH=src ./train.py --dataset /content/drive/My Drive/writrly_proj_files/Atlantic_essays/health --combine 500 --run_name 'atlantic_health_concat_345' --model_name '345M'

### Save our checkpoints to start training again later

In [0]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

Load your trained model for use in sampling below (117M or 345M)

In [0]:
!cp -r /content/gpt-2/checkpoint/NR_train_personal/* /content/gpt-2/models/117M_NR/

cp: target '/content/gpt-2/models/117M_NR/' is not a directory


In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "117M"

W0620 16:35:08.869527 140272897111936 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0620 16:35:09.208445 140272897111936 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:55: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-06-20 16:35:09.210092: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-06-20 16:35:09.252310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-20 16:35:09.252799: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2019-06-20 16:35:09.253079: I tensorflow/stream_executor/pl

To check flag descriptions, use:

In [0]:
!python3 src/interactive_conditional_samples.py -- --help

W0620 16:30:15.744753 139747760654208 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

Type:        function
String form: <function interact_model at 0x7f198d753d08>
File:        /content/gpt-2/src/interactive_conditional_samples.py
Line:        11
Docstring:   Interactively run the model
:model_name=117M : String, which model to use
:seed=None : Integer seed for random number generators, fix seed to reproduce
 results
:nsamples=1 : Number of samples to return total
:batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
:length=None : Number of tokens in generated text, if None (default), is
 determined by model hyperparameters
:temperature=1 : Float value controlling randomness in boltzmann
 distribution. Lower temperature results in less random completions. As the
 temperature approaches zero, the model will become deterministic and
 repetitive. Higher te

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [0]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

To check flag descriptions, use:

In [0]:
!python3 src/generate_unconditional_samples.py -- --help